Importing all the necessary libraries for the ELT

In [1]:
import pandas as pd

In [2]:
import pyorc

In [3]:
from datetime import datetime

Read file

In [4]:
example = open("trades.orc", 'rb')

In [5]:
reader = pyorc.Reader(example)

In [6]:
schema = reader.schema

In [7]:
columns = schema.fields

In [8]:
df=pd.DataFrame(reader, columns=columns, dtype=str)

In [9]:
df

,trade_id,price,quantity,time,isSelling
0,1117732504.0,1793.61,0.0201,1680220800001.0,False
1,1117732505.0,1793.6,0.0197,1680220800001.0,True
2,1117732506.0,1793.61,0.038,1680220800005.0,False
3,1117732507.0,1793.6,0.0222,1680220800005.0,True
4,1117732508.0,1793.61,0.0209,1680220800005.0,False
...,...,...,...,...,...
902950,1118635454.0,1821.51,0.0061,1680307199996.0,True
902951,1118635455.0,1821.51,0.0153,1680307199997.0,True
902952,1118635456.0,1821.51,0.0341,1680307199997.0,True
902953,1118635457.0,1821.51,0.033,1680307199999.0,True


In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902955 entries, 0 to 902954
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   trade_id   902954 non-null  object
 1   price      902954 non-null  object
 2   quantity   902954 non-null  object
 3   time       902954 non-null  object
 4   isSelling  902954 non-null  object
dtypes: object(5)
memory usage: 34.4+ MB
None


Prepare the DataFreame


In [11]:
df = df.rename(columns={'trade_id': 'trade_id_src'})

In [12]:
df = df.rename(columns={'price': 'price_src'})

In [13]:
df = df.rename(columns={'quantity': 'quantity_src'})

In [14]:
df = df.rename(columns={'time': 'time_src'})

In [15]:
df = df.rename(columns={'isSelling': 'isSelling_src'})

Clean and customize

In [16]:
df['trade_id_str'] = df['trade_id_src'].str.slice(stop=-2)

In [17]:
df['price_fl'] = df['price_src'].astype(float)

In [18]:
df['quantity_fl'] = df['quantity_src'].astype(float)

In [19]:
df['time_dt'] = pd.to_datetime(df['time_src'].astype(float)/1000, unit='s')


In [20]:
df['isSelling_bl'] = df['isSelling_src'].str.lower() == 'true'

In [21]:
df = df.sort_values('time_dt')

In [22]:
df = df.dropna()

In [23]:
df = df.drop_duplicates(subset=['price_fl', 'quantity_fl', 'time_dt', 'isSelling_bl' ], keep='first')

In [24]:
df

,trade_id_src,price_src,quantity_src,time_src,isSelling_src,trade_id_str,price_fl,quantity_fl,time_dt,isSelling_bl
0,1117732504.0,1793.61,0.0201,1680220800001.0,False,1117732504,1793.61,0.0201,2023-03-31 00:00:00.000999936,False
1,1117732505.0,1793.6,0.0197,1680220800001.0,True,1117732505,1793.60,0.0197,2023-03-31 00:00:00.000999936,True
2,1117732506.0,1793.61,0.038,1680220800005.0,False,1117732506,1793.61,0.0380,2023-03-31 00:00:00.005000192,False
3,1117732507.0,1793.6,0.0222,1680220800005.0,True,1117732507,1793.60,0.0222,2023-03-31 00:00:00.005000192,True
4,1117732508.0,1793.61,0.0209,1680220800005.0,False,1117732508,1793.61,0.0209,2023-03-31 00:00:00.005000192,False
...,...,...,...,...,...,...,...,...,...,...
902950,1118635454.0,1821.51,0.0061,1680307199996.0,True,1118635454,1821.51,0.0061,2023-03-31 23:59:59.996000000,True
902951,1118635455.0,1821.51,0.0153,1680307199997.0,True,1118635455,1821.51,0.0153,2023-03-31 23:59:59.996999936,True
902952,1118635456.0,1821.51,0.0341,1680307199997.0,True,1118635456,1821.51,0.0341,2023-03-31 23:59:59.996999936,True
902953,1118635457.0,1821.51,0.033,1680307199999.0,True,1118635457,1821.51,0.0330,2023-03-31 23:59:59.999000064,True


In [25]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 888619 entries, 0 to 902954
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   trade_id_src   888619 non-null  object        
 1   price_src      888619 non-null  object        
 2   quantity_src   888619 non-null  object        
 3   time_src       888619 non-null  object        
 4   isSelling_src  888619 non-null  object        
 5   trade_id_str   888619 non-null  object        
 6   price_fl       888619 non-null  float64       
 7   quantity_fl    888619 non-null  float64       
 8   time_dt        888619 non-null  datetime64[ns]
 9   isSelling_bl   888619 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), object(6)
memory usage: 68.6+ MB
None


Index and Audit fields

In [26]:
df['last_modified_dt'] = datetime.now()

In [27]:
df = df.set_index('trade_id_str')

In [28]:
df

,trade_id_src,price_src,quantity_src,time_src,isSelling_src,price_fl,quantity_fl,time_dt,isSelling_bl,last_modified_dt
trade_id_str,,,,,,,,,,
1117732504,1117732504.0,1793.61,0.0201,1680220800001.0,False,1793.61,0.0201,2023-03-31 00:00:00.000999936,False,2023-04-15 16:13:17.783043
1117732505,1117732505.0,1793.6,0.0197,1680220800001.0,True,1793.60,0.0197,2023-03-31 00:00:00.000999936,True,2023-04-15 16:13:17.783043
1117732506,1117732506.0,1793.61,0.038,1680220800005.0,False,1793.61,0.0380,2023-03-31 00:00:00.005000192,False,2023-04-15 16:13:17.783043
1117732507,1117732507.0,1793.6,0.0222,1680220800005.0,True,1793.60,0.0222,2023-03-31 00:00:00.005000192,True,2023-04-15 16:13:17.783043
1117732508,1117732508.0,1793.61,0.0209,1680220800005.0,False,1793.61,0.0209,2023-03-31 00:00:00.005000192,False,2023-04-15 16:13:17.783043
...,...,...,...,...,...,...,...,...,...,...
1118635454,1118635454.0,1821.51,0.0061,1680307199996.0,True,1821.51,0.0061,2023-03-31 23:59:59.996000000,True,2023-04-15 16:13:17.783043
1118635455,1118635455.0,1821.51,0.0153,1680307199997.0,True,1821.51,0.0153,2023-03-31 23:59:59.996999936,True,2023-04-15 16:13:17.783043
1118635456,1118635456.0,1821.51,0.0341,1680307199997.0,True,1821.51,0.0341,2023-03-31 23:59:59.996999936,True,2023-04-15 16:13:17.783043


In [29]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 888619 entries, 1117732504 to 1118635458
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   trade_id_src      888619 non-null  object        
 1   price_src         888619 non-null  object        
 2   quantity_src      888619 non-null  object        
 3   time_src          888619 non-null  object        
 4   isSelling_src     888619 non-null  object        
 5   price_fl          888619 non-null  float64       
 6   quantity_fl       888619 non-null  float64       
 7   time_dt           888619 non-null  datetime64[ns]
 8   isSelling_bl      888619 non-null  bool          
 9   last_modified_dt  888619 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(2), object(5)
memory usage: 68.6+ MB
None


Find Metrics

In [30]:
df['total'] = df.apply(lambda row: -1 if row['isSelling_bl'] == False else 1, axis=1) * df['quantity_fl'].values * df['price_fl']

In [31]:
bot_outcome = df['total'].sum()

In [32]:
df['rowcount'] = df['isSelling_bl'].where(df['isSelling_bl'], 0).astype(int).groupby((df['isSelling_bl'] != df['isSelling_bl'].shift()).cumsum()).cumsum()

In [33]:
longest_chain = pd.to_numeric(df['rowcount'].max())

In [34]:
max_index = pd.to_numeric(df['rowcount'].idxmax())

In [35]:
min_index = (max_index-longest_chain).astype(str)

In [36]:
max_index = max_index.astype(str)

In [37]:
chain_price_diff = (df.loc[max_index, 'quantity_fl']*df.loc[max_index, 'price_fl'])-(df.loc[min_index, 'quantity_fl']*df.loc[min_index, 'price_fl'])

Results

In [39]:
df

,trade_id_src,price_src,quantity_src,time_src,isSelling_src,price_fl,quantity_fl,time_dt,isSelling_bl,last_modified_dt,total,rowcount
trade_id_str,,,,,,,,,,,,
1117732504,1117732504.0,1793.61,0.0201,1680220800001.0,False,1793.61,0.0201,2023-03-31 00:00:00.000999936,False,2023-04-15 16:13:17.783043,-36.051561,0
1117732505,1117732505.0,1793.6,0.0197,1680220800001.0,True,1793.60,0.0197,2023-03-31 00:00:00.000999936,True,2023-04-15 16:13:17.783043,35.333920,1
1117732506,1117732506.0,1793.61,0.038,1680220800005.0,False,1793.61,0.0380,2023-03-31 00:00:00.005000192,False,2023-04-15 16:13:17.783043,-68.157180,0
1117732507,1117732507.0,1793.6,0.0222,1680220800005.0,True,1793.60,0.0222,2023-03-31 00:00:00.005000192,True,2023-04-15 16:13:17.783043,39.817920,1
1117732508,1117732508.0,1793.61,0.0209,1680220800005.0,False,1793.61,0.0209,2023-03-31 00:00:00.005000192,False,2023-04-15 16:13:17.783043,-37.486449,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1118635454,1118635454.0,1821.51,0.0061,1680307199996.0,True,1821.51,0.0061,2023-03-31 23:59:59.996000000,True,2023-04-15 16:13:17.783043,11.111211,1
1118635455,1118635455.0,1821.51,0.0153,1680307199997.0,True,1821.51,0.0153,2023-03-31 23:59:59.996999936,True,2023-04-15 16:13:17.783043,27.869103,2
1118635456,1118635456.0,1821.51,0.0341,1680307199997.0,True,1821.51,0.0341,2023-03-31 23:59:59.996999936,True,2023-04-15 16:13:17.783043,62.113491,3


In [40]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 888619 entries, 1117732504 to 1118635458
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   trade_id_src      888619 non-null  object        
 1   price_src         888619 non-null  object        
 2   quantity_src      888619 non-null  object        
 3   time_src          888619 non-null  object        
 4   isSelling_src     888619 non-null  object        
 5   price_fl          888619 non-null  float64       
 6   quantity_fl       888619 non-null  float64       
 7   time_dt           888619 non-null  datetime64[ns]
 8   isSelling_bl      888619 non-null  bool          
 9   last_modified_dt  888619 non-null  datetime64[ns]
 10  total             888619 non-null  float64       
 11  rowcount          888619 non-null  int32         
dtypes: bool(1), datetime64[ns](2), float64(3), int32(1), object(5)
memory usage: 111.1+ MB
None


In [38]:
print('Bot outcome for the day:', bot_outcome)
print('Longest chain of consecutive sells:', longest_chain)
print('Price difference between first and last operation of longest chain:', chain_price_diff)

Bot outcome for the day: -41614981.018217005
Longest chain of consecutive sells: 485
Price difference between first and last operation of longest chain: 4765.926670000001
